In [ ]:
!curl -fsSL https://tailscale.com/install.sh | sh

# Starten von Tailscale mit Ihrem Authentifizierungstoken
!tailscale up --authkey=YOUR_AUTH_KEY

# Prüfen Sie den Status von Tailscale
!tailscale status

In [ ]:
import json
import pandas as pd
import tensorflow as tf
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import io

# 1. Laden der Konfiguration
with open('config.json') as config_file:
    config = json.load(config_file)

data_url = config['data_url']
features = config['features']
labels = config['labels']
model_output_path = config['model_output_path']
epochs = config['parameters']['epochs']
batch_size = config['parameters']['batch_size']

# 2. Daten laden
data = pd.read_csv(data_url)
X = data[features]
y = data[labels]

# 3. Modell definieren und trainieren
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(features),)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=epochs, batch_size=batch_size)

# 4. Modell auf Google Drive speichern
def upload_model_to_drive(file_path, file_name):
    scopes = ['https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('service_account_credentials.json', scopes)
    drive_service = build('drive', 'v3', credentials=creds)
    
    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, mimetype='application/octet-stream')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print(f'Model uploaded to Drive with ID: {file.get("id")}')
    return file.get('id')

model.save(model_output_path)
model_file_id = upload_model_to_drive(model_output_path, config['model_output_path'])


In [ ]:
import subprocess

def trigger_pull_script_via_ssh(ssh_config, model_file_id):
    ssh_command = f"ssh -i {ssh_config['key_file']} {ssh_config['user']}@{ssh_config['server']} 'MODEL_FILE_ID={model_file_id} python3 {ssh_config['script_path']}'"
    try:
        result = subprocess.run(ssh_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("SSH command executed successfully:", result.stdout.decode())
    except subprocess.CalledProcessError as e:
        print("SSH command failed:", e.stderr.decode())
        log_error_to_drive(model_file_id, e.stderr.decode())

# Fehlerprotokollierungsfunktion mit Speicherung auf Google Drive
def log_error_to_drive(model_file_id, error_message):
    log_file_path = '/content/failed_upload_log.txt'
    with open(log_file_path, 'a') as log_file:
        log_file.write(f"Failed to execute pull_models.py for model ID: {model_file_id}\n")
        log_file.write(f"Error: {error_message}\n")
        log_file.write(f"Model is saved in Google Drive with ID: {model_file_id}\n\n")
    print(f"Error logged in {log_file_path}")

    # Hochladen der Logdatei auf Google Drive
    upload_model_to_drive(log_file_path, 'failed_upload_log.txt')

# Überprüfen, ob der SSH-Trigger aktiviert ist
if config['ssh_config']['trigger_pull_script']:
    trigger_pull_script_via_ssh(config['ssh_config'], model_file_id)